In [1]:
import requests
import bs4
import pandas as pd
import re
import time

import warnings
import urllib3

# Disabilita gli avvisi di HTTPS non verificato
urllib3.disable_warnings()

# Ignora i FutureWarning
warnings.simplefilter(action="ignore", category=FutureWarning)
from playsound import playsound

pd.set_option('display.max_columns', None)
informazioni_geografiche = pd.read_csv("informazioni_geografiche.csv")

In [ ]:
def raccolta(comune, pagina_del_comune):
    df_dic = {}

    def pulisci_dataframe(data_dict, colonne_da_rinominare):
        """
        Pulisce un DataFrame disordinato, aggiungendo automaticamente la prima riga 
        e rinominando le colonne specificate.

        :param data_dict: Dizionario dei dati grezzi
        :param colonne_da_rinominare: Lista di nomi per le colonne finali
        :return: DataFrame pulito
        """
        
        # Convertiamo il dizionario in un DataFrame temporaneo
        df_temp = pd.DataFrame(data_dict)

        # Recuperiamo automaticamente la prima riga (nomi effettivi delle colonne)
        prima_riga = [col.replace(":", "").strip() for col in df_temp.columns[:len(colonne_da_rinominare)]]
        
        # Creiamo un nuovo DataFrame con la prima riga inclusa
        df_clean = pd.DataFrame({
            colonne_da_rinominare[i]: [prima_riga[i]] + df_temp.iloc[:, i].str.replace(":", "").str.strip().tolist()
            for i in range(len(colonne_da_rinominare))
        })

        # Pulizia e conversione dei numeri
        for col in df_clean.columns:
            if col == "Percentuale_Totale": #.str.contains("%").any():  # Se contiene percentuali
                df_clean[col] = df_clean[col].str.replace(",", ".").str.rstrip("%").astype(float) / 10
            elif df_clean[col].str.replace(".", "", regex=False).str.isnumeric().all():  # Se è un numero
                df_clean[col] = df_clean[col].str.replace(".", "", regex=False).astype(int)  # Rimuove i punti

        return df_clean

    def transform_dict_to_df(data_dict):
        df = pd.DataFrame(data_dict)
        new_data = {}

        for col in df.columns:
            if col != list(data_dict.keys())[0]:  
                for idx, age_group in df[list(data_dict.keys())[0]].items():
                    new_col_name = f"{col}_{age_group}"
                    new_data[new_col_name] = df.at[idx, col]  # Aggiungiamo il valore corrispondente

        # Creiamo il nuovo DataFrame con una sola riga
        new_df = pd.DataFrame([new_data])
        
        return new_df

    def concat_dataframe(accesso, df2):
        df1 = pd.read_csv(accesso)
        df = pd.concat([df1, df2], ignore_index=True)
        df.to_csv(accesso, index=False)

        return df

    # Trova il paragrafo con le informazioni
    dati_paragrafo = pagina_del_comune.find("p", class_="well")

    if dati_paragrafo:
        testo = dati_paragrafo.get_text()

        # Estrai le informazioni con slicing testuale
        info = {}
        info["Municipio"] = testo.split("Municipio: ")[1].split(",")[0]
        info["Provincia"] = testo.split("Provincia: ")[1].split(".")[0]
        info["Distanza dal capoluogo"] = testo.split("Distanza dal capoluogo")[1].split(": ")[1].split(" ")[0] + " Km"
        info["Abitanti"] = testo.split("Abitanti: ")[1].split(" ")[0]
        info["Denominazione"] = testo.split("Denominazione: ")[1].split(".")[0]

        # Stampare il risultato
        for k, v in info.items():
            df_dic[k] = v

    else:
        print("Errore: Impossibile trovare il paragrafo con i dati richiesti.")

    # Trova il paragrafo che contiene la densità abitativa
    densita_text = None
    for p in pagina_del_comune.find_all("p"):
        if "Densità abitativa" in p.text:
            densita_text = p.text
            break

    # Estrai il numero dalla stringa trovata
    if densita_text:
        match = re.search(r"Densità abitativa:\s*([0-9.,]+)", densita_text)
        if match:
            densita = match.group(1)
            #print(f"Densità abitativa: {densita} abitanti per km²")
            df_dic['Densità abitativa'] = densita
        else:
            print("Densità abitativa non trovata.")
    else:
        print("Sezione sulla densità abitativa non trovata.")
    
    print(df_dic)
    df_comune = pd.DataFrame([df_dic])

    # Cerca tutte le <div> che contengono tabelle (le tabelle sono racchiuse in div con classe "tb_resp")
    div_tables = pagina_del_comune.find_all("div", class_="tb_resp")

    families_table = None
    for div in div_tables:
        table = div.find("table")
        if table:
            # Verifica se la prima riga (con classe "info") contiene "Numero di componenti"
            header_row = table.find("tr", class_="info")
            if header_row:
                cells = header_row.find_all("td")
                if cells and "Numero di componenti" in cells[0].get_text():
                    families_table = table
                    break

    if families_table:
        # Estrazione delle righe della tabella
        rows = []
        for tr in families_table.find_all("tr"):
            cols = [td.get_text(strip=True) for td in tr.find_all("td")]
            rows.append(cols)
        
        # La struttura della tabella è la seguente:
        # Prima riga (header): ["Numero di componenti", "1", "2", "3", "4", "5", "6 o più"]
        # Seconda riga (dati): ["Numero di famiglie", "391", "390", "244", "156", "50", "32"]
        header = rows[0]
        data = rows[1]
        
        # Creare il DataFrame con la prima riga come intestazione
        df = pd.DataFrame([data], columns=header)
        df.rename(columns={"6 o piÃ¹": "6 o più"}, inplace=True)
        
        #print("Tabella 'Famiglie e loro numerosità di componenti':")
        #print(df)
        
        # Salva il risultato in un file CSV
        #df.to_csv("famiglie_solarolo.csv", index=False)
    else:
        print("Tabella 'Famiglie e loro numerosità di componenti' non trovata.")

    famiglie = df.drop(columns=["Numero di componenti"])
    famiglie["Comune"] = comune
    concat_dataframe('inf_famiglie.csv', famiglie)

    def table(title):
        # Trova il titolo della sezione desiderata
        titolo_sezione = pagina_del_comune.find("span", id="accatre", string=title)
        # Dati sulla popolazione straniera residente a Solarolo

        if titolo_sezione:
            # Trova la tabella che segue il titolo della sezione
            tabella = titolo_sezione.find_next("table", class_="table")

            if tabella:
                # Estrai le intestazioni
                intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

                # Estrai i dati della tabella
                dati = []
                for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                    colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                    dati.append(colonne)

                # Creazione del DataFrame
                df = pd.DataFrame(dati, columns=intestazioni)
                
                # Salva il DataFrame in un file CSV
                #df.to_csv("popolazione_straniera_solarolo.csv", index=False, encoding="utf-8")

                #print("File CSV creato con successo: popolazione_straniera_solarolo.csv")
            else:
                print("Errore: Impossibile trovare la tabella della popolazione straniera.")
        else:
            print("Errore: Impossibile trovare la sezione 'Dati sulla popolazione straniera residente a Solarolo'.")
        
        return df

    colonne_finali = ["Settore", "Numero_Aziende", "Addetti", "Percentuale_Totale"]
    settori = pulisci_dataframe(table("Geografia, Anagrafe e Statistica").to_dict(), colonne_finali)
    settori["Comune"] = comune
    concat_dataframe('inf_settori.csv', settori)


    eta = transform_dict_to_df(table(f"Dati sulla popolazione residente a {comune}")
                        .rename(columns={"EtÃ": "Età"})
                        .to_dict())
    eta["Comune"] = comune
    concat_dataframe('inf_età.csv', eta)

    stranieri = transform_dict_to_df(table(f"Dati sulla popolazione straniera residente a {comune}")
                        .rename(columns={"EtÃ": "Età"})
                        .to_dict())

    for x in list(stranieri.columns):
        nuovo = str(x).replace("PiÃ¹", "Più")
        stranieri.rename(columns={x: nuovo}, inplace=True)  # Modifica in-place

    stranieri["Comune"] = comune
    concat_dataframe('inf_stranieri.csv', stranieri)

    scolarizzazione = transform_dict_to_df(table(f"Livelli di scolarizzazione a {comune}")
                        .to_dict())
    scolarizzazione["Comune"] = comune
    concat_dataframe('inf_scolarizzazione.csv', scolarizzazione)

    def correzione_moneta(valore):
        valore = str(valore).replace("â¬", "€")
        return valore

    def table(title):
        # Trova il titolo della sezione desiderata
        titolo_sezione = pagina_del_comune.find("h4", string=title)

        if titolo_sezione:
            # Trova la tabella che segue il titolo della sezione
            tabella = titolo_sezione.find_next("table", class_="table")

            if tabella:
                # Estrai le intestazioni
                intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

                # Estrai i dati della tabella
                dati = []
                for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                    colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                    dati.append(colonne)

                # Creazione del DataFrame
                df = pd.DataFrame(dati, columns=intestazioni)
                df["Comune"] = comune

                if "Reddito" in df.columns:
                    df["Reddito"] = df["Reddito"].apply(correzione_moneta)
                    df["Media annuale"] = df["Media annuale"].apply(correzione_moneta)
                    df["Media mensile"] = df["Media mensile"].apply(correzione_moneta)
                    df["Anno precedente"] = df["Anno precedente"].apply(correzione_moneta)
                    df["Variazione"] = df["Variazione"].apply(correzione_moneta)

                # Salva il DataFrame in un file CSV
                #df.to_csv("edifici_solarolo_costruzione.csv", index=False, encoding="utf-8")

                #print("File CSV creato con successo: edifici_solarolo_costruzione.csv")
            else:
                print("Errore: Impossibile trovare la tabella degli edifici per data di costruzione.")
        else:
            print("Errore: Impossibile trovare la sezione 'Gli edifici a Solarolo per data di costruzione'.")
        
        return df

    cronologia_edifici = table(f"Gli edifici a {comune} per data di costruzione").drop(columns=["Date"])
    cronologia_edifici["Comune"] = comune
    concat_dataframe('inf_cronologia_edifici.csv', cronologia_edifici)

    numero_di_piani = table(f"Gli edifici a {comune} per numero di piani").drop(columns=["Numero di piani"])
    numero_di_piani.rename(columns={"Quattro o piÃ¹": "Quattro o più"}, inplace=True)
    numero_di_piani["Comune"] = comune
    concat_dataframe('inf_numero_di_piani.csv', numero_di_piani)

    interni = table(f"Gli edifici a {comune} per numero di interni").drop(columns=["Numero di interni"])
    interni["Comune"] = comune
    concat_dataframe('inf_interni.csv', interni)

    redditi_01 = table("Redditi e contribuenti per tipo di reddito")#.to_csv("inf_redditi_01.csv", index=False)
    concat_dataframe('inf_redditi_01.csv', redditi_01)

    redditi_02 = table("Redditi, imposte e addizionali comunali e regionali")#.to_csv("inf_redditi_02.csv", index=False)
    concat_dataframe('inf_redditi_02.csv', redditi_02)

    redditi_03 = table("Redditi e contribuenti per fasce di reddito")#.to_csv("inf_redditi_03.csv", index=False)
    concat_dataframe('inf_redditi_03.csv', redditi_03)

    # Trova il titolo della sezione con una ricerca flessibile
    titolo_sezione = None
    for tag in pagina_del_comune.find_all(["h2", "h3", "span", "p"]):  # Controlliamo più tag possibili
        if re.search(r"Parrocchie", tag.get_text(), re.IGNORECASE):
            titolo_sezione = tag
            break

    if titolo_sezione:
        # Trova la tabella che segue il titolo della sezione
        tabella = titolo_sezione.find_next("table")

        if tabella:
            # Estrai le intestazioni della tabella
            intestazioni = [th.get_text(strip=True) for th in tabella.find("tr").find_all("td")]

            # Estrai i dati
            dati = []
            for riga in tabella.find_all("tr")[1:]:  # Ignora la prima riga che contiene le intestazioni
                colonne = [td.get_text(strip=True) for td in riga.find_all("td")]
                dati.append(colonne)

            # Creazione del DataFrame
            parrocchie = pd.DataFrame(dati, columns=intestazioni)

            # Salva il DataFrame in un file CSV
            #df.to_csv("parrocchie_solarolo.csv", index=False, encoding="utf-8")

            #print("✅ File CSV creato con successo: parrocchie_solarolo.csv")
        else:
            print("❌ Errore: Impossibile trovare la tabella delle parrocchie.")
    else:
        print("❌ Errore: Non è stata trovata la sezione 'Parrocchie'.")
    
    concat_dataframe('inf_parrocchie.csv', parrocchie)

    # Trova la sezione delle scuole
    scuole_section = pagina_del_comune.find("a", {"name": "scuole"})

    if scuole_section:
        table = scuole_section.find_next("table")  # Trova la tabella successiva

        # Estrarre tutte le righe della tabella
        rows = []
        for tr in table.find_all("tr"):
            cells = [td.text.strip() for td in tr.find_all(["th", "td"])]
            rows.append(cells)

        # Se la tabella ha almeno una riga, usa la prima come intestazione
        if len(rows) > 1:
            headers = rows[0]  # Prima riga come intestazione
            data = rows[1:]     # Dati senza la prima riga
        else:
            headers = None
            data = rows

        # Creare un DataFrame Pandas
        scuole = pd.DataFrame(data, columns=headers if headers else None)

        # Stampare il risultato
        #print(df)
    else:
        print("Tabella delle scuole non trovata.")
    
    concat_dataframe('inf_scuole.csv', scuole)

    informazioni_geografiche = pd.read_csv("informazioni_geografiche.csv")

    data = eta.to_dict()

    # Intervalli di età e relativi punti medi
    age_intervals = {
        '0-4': 2, '5-9': 7, '10-14': 12, '15-19': 17, '20-24': 22, '25-29': 27,
        '30-34': 32, '35-39': 37, '40-44': 42, '45-49': 47, '50-54': 52, '55-59': 57,
        '60-64': 62, '65-69': 67, '70-74': 72, '>74': 77  # Il gruppo ">74" usa 77 come stima
    }

    # Calcolare la somma ponderata per la popolazione totale
    total_population = 0
    weighted_sum = 0

    # Calcolare somma ponderata e totale della popolazione
    for key, value in data.items():
        if 'Totale' in key:  # Consideriamo solo i totali
            age_group = key.split('_')[0]  # Estrai il gruppo di età (es. '0-4')
            middle_age = age_intervals[age_group]  # Età media stimata per il gruppo
            total_population += int(value[0])  # Converti in intero
            weighted_sum += middle_age * int(value[0])  # Aggiungi il contributo ponderato

    # Calcolare la media di età
    average_age = round(weighted_sum / total_population, 2)



    # Aggiungi il nome del comune
    df_comune["Comune"] = comune
    #concat_dataframe('inf_comunali.csv', df_comune)

    # Estrai i dati direttamente dalle colonne specifiche
    df_comune["Regione"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["Regione"].iloc[0]
    df_comune["Zona"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["Macro Zona della Regione"].iloc[0]
    df_comune["sigla_provincia"] = informazioni_geografiche[informazioni_geografiche["Comune"] == comune]["sigla_provincia"].iloc[0]
    df_comune["Età (Media)"] = average_age

    # Aggiungi il numero di parrocchie e scuole
    df_comune["Parrocchie"] = parrocchie.shape[0]
    df_comune["Scuole"] = scuole.shape[0]

    # Estrai le percentuali di addetti dai settori
    df_comune["Industria Addetti"] = f'{round(settori[settori["Settore"] == "Industrie"]["Percentuale_Totale"].iloc[0], 2)}%'
    df_comune["Servizi Addetti"] = f'{round(settori[settori["Settore"] == "Servizi"]["Percentuale_Totale"].iloc[0], 2)}%'
    df_comune["Amministrazione Addetti"] = f'{round(settori[settori["Settore"] == "Amministrazione"]["Percentuale_Totale"].iloc[0], 2)}%'
    df_comune["Altro Addetti"] = f'{round(settori[settori["Settore"] == "Altro"]["Percentuale_Totale"].iloc[0], 2)}%'

    # Estrai i dati sulla scolarizzazione
    df_comune["Diplomati"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Diploma_Totale"].iloc[0]
    df_comune["Laureati"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Laurea_Totale"].iloc[0]
    df_comune["Liscenziati Midia Totale"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Licenza Media_Totale"].iloc[0]
    df_comune["Licenza Elementare Totale"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Licenza Elementare_Totale"].iloc[0]
    df_comune["Alfabeti"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Alfabeti_Totale"].iloc[0]
    df_comune["Analfabeti"] = scolarizzazione[scolarizzazione["Comune"] == comune]["Analfabeti_Totale"].iloc[0]

    # Redditi
    df_comune["Media annuale (Redditi)"] = round(redditi_01['Media annuale'].str.replace("€", "").str.replace(".", "")
                            .str.replace(",", ".").astype(float).mean(), 2)

    df_comune["Media Mensile (Redditi)"] = round(redditi_01['Media mensile'].str.replace("€", "").str.replace(".", "")
                            .str.replace(",", ".").astype(float).mean(), 2)

    df_comune["Imposta netta annuale"] = float(redditi_02.loc[redditi_02["Categoria"] == "Imposta netta", "Media annuale"]
                                                .iloc[0].replace("€", "").replace(".", "").replace(",", ".").strip())

    df_comune["Imposta netta mensile"] = float(redditi_02.loc[redditi_02["Categoria"] == "Imposta netta", "Media mensile"]
                                                .iloc[0].replace("€", "").replace(".", "").replace(",", ".").strip())

    # Calcola il totale di stranieri
    df_comune["Totale di stranieri"] = (int(stranieri[stranieri["Comune"] == comune]['Totale_Da 0 a 29 anni'].iloc[0]) 
                                        + int(stranieri[stranieri["Comune"] == comune]['Totale_Da 30 a 54 anni'].iloc[0])
                                        + int(stranieri[stranieri["Comune"] == comune]['Totale_Più di 54 anni'].iloc[0]))
    
    #display(df_comune)
    
    concat_dataframe('inf_comunali.csv', df_comune)

In [6]:
gia_analizzati = pd.read_csv('inf_comunali.csv')
errori = pd.read_csv('Errori.csv')

errori_da_rimuovere = list(errori['Comuni'].unique())  # Lista dei comuni unici

# Filtra i comuni non ancora analizzati
comuni_da_analizzare = informazioni_geografiche[
    (~informazioni_geografiche['Comune'].isin(gia_analizzati['Comune'].unique())) &
    (~informazioni_geografiche['Comune'].isin(errori_da_rimuovere))
]

print(f'Totale di comuni analizzati: {len(list(gia_analizzati['Comune'].unique()))}')
print(f'Totale di comuni da analizzare: {comuni_da_analizzare.shape[0]}\n')

# Seleziona solo i primi 10
comuni_da_analizzare = comuni_da_analizzare.iloc[:50]  # Oppure .head(10)f

# Crea il dizionario con il Comune come chiave e la Regione come valore
comuni_da_analizzare = dict(zip(comuni_da_analizzare['Comune'], comuni_da_analizzare['Regione']))

print(comuni_da_analizzare)
print(list(comuni_da_analizzare.keys())[0:60])
#print(comuni_da_analizzare["L'Aquila"])
print(len(list(comuni_da_analizzare.keys())))

#Totale di comuni analizzati: 97 | 6088 6133
#Totale di comuni da analizzare: 7805 | 1300 1252
# 'San Venanzo', 'Castel Volturno', 'Poggio San Vicino', 'Montemarciano'

Totale di comuni analizzati: 6133
Totale di comuni da analizzare: 1252

{'Fiastra': 'Marche', 'Bitetto': 'Puglia', 'Lappano': 'Calabria', 'Castelvetro di Modena': 'Emilia-Romagna', 'Caldonazzo': 'Trentino-Alto Adige', 'Mele': 'Liguria', 'Gavi': 'Piemonte', 'San Pietro Avellana': 'Molise', 'Monsampietro Morico': 'Marche', 'Trana': 'Piemonte', 'Camerota': 'Campania', 'Crevoladossola': 'Piemonte', 'Cortemaggiore': 'Emilia-Romagna', 'Castel del Rio': 'Emilia-Romagna', 'Terricciola': 'Toscana', 'Mirano': 'Veneto', 'Mirandola': 'Emilia-Romagna', 'Campo di Trens': 'Trentino-Alto Adige', 'Albenga': 'Liguria', 'Vernante': 'Piemonte', 'Copparo': 'Emilia-Romagna', 'Sona': 'Veneto', 'Casola in Lunigiana': 'Toscana', 'Francavilla Angitola': 'Calabria', 'Montefelcino': 'Marche', 'Serramazzoni': 'Emilia-Romagna', 'Brennero': 'Trentino-Alto Adige', 'Quiliano': 'Liguria', 'Craco': 'Basilicata', 'Salandra': 'Basilicata', 'Rittana': 'Piemonte', 'Pradleves': 'Piemonte', 'San Leo': 'Emilia-Romagna', 'Garba

In [ ]:
#comuni_da_analizzare = {"Solarolo": "Emilia-Romagna"}
#comuni_da_analizzare = {"Solarolo": "Emilia-Romagna", "Civita": "Calabria", "Faenza": "Emilia-Romagna", "Bologna": "Emilia-Romagna", "Ravenna": "Emilia-Romagna"}
#comuni_da_analizzare = {"L'Aquila": "Abruzzo"}
errori_raccolta = []
cancella = comuni_da_analizzare

import os

for comune_dic, regione_dic in comuni_da_analizzare.items():
    url_comune = re.sub(r"[-' ]+", "", comune_dic.lower())
    url_regione = re.sub(r"[-' ]+", "", regione_dic.lower())
    url = f'https://italia.indettaglio.it/ita/{url_regione}/{url_comune}.html'
    comune = comune_dic

    try:
        # Inizio del calcolo del tempo
        #start_time = time.time()

        # Effettua la richiesta HTTP
        requisicao = requests.get(url, verify=False, timeout=10)  # Timeout per evitare blocchi infiniti
        pagina_del_comune = bs4.BeautifulSoup(requisicao.text, "html.parser")

        # Calcolo del tempo impiegato
        #end_time = time.time()
        #elapsed_time = end_time - start_time
        #print(f"⏱️ Tempo di richiesta per {comune}: {elapsed_time:.2f} secondi")
    except Exception as e:
        print(f"🚨 Errore nella richiesta per {comune}: {e}")
        errori_raccolta.append(comune)  # Salva il comune con errore
        continue  # Salta al comune successivo

    try:
        # Inizio del calcolo del tempo per la raccolta dei dati
        #start_time_raccolta = time.time()

        # Tenta la raccolta dei dati
        raccolta(comune, pagina_del_comune)
        informazioni_geografiche.loc[informazioni_geografiche["Comune"] == comune, "Raccolto"] = 1
        print(f"✅ {comune} - Raccolta completata")

        # Calcolo del tempo impiegato per la raccolta
        #end_time_raccolta = time.time()
        #elapsed_time_raccolta = end_time_raccolta - start_time_raccolta
        #print(f"⏱️ Tempo di raccolta per {comune}: {elapsed_time_raccolta:.2f} secondi")
    except Exception as e:
        informazioni_geografiche.loc[informazioni_geografiche["Comune"] == comune, "Raccolto"] = 0
        errori_raccolta.append(comune)
        print(f"❌ Errore nella funzione RACCOLTA per {comune}: {e}")
        continue

    # Assicura il salvataggio del CSV a ogni iterazione
    try:
        informazioni_geografiche.to_csv("informazioni_geografiche.csv", index=False)
    except Exception as e:
        print(f"⚠️ Errore nel salvataggio del file CSV: {e}")

    time.sleep(0.5)  # Pausa per evitare di sovraccaricare il server

# **Gestione del file Errori.csv per aggiungere nuovi errori senza sovrascrivere**
errori_path = "Errori.csv"

# Se esiste già, lo carica, altrimenti crea un DataFrame vuoto
if os.path.exists(errori_path):
    errori_esistenti = pd.read_csv(errori_path)
else:
    errori_esistenti = pd.DataFrame(columns=["Comuni"])  # Usa "Comuni" come nome di colonna

# Crea DataFrame con i nuovi errori
errori_raccolta_df = pd.DataFrame({"Comuni": errori_raccolta})

# Unisce i nuovi errori con quelli esistenti
errori_totali = pd.concat([errori_esistenti, errori_raccolta_df], ignore_index=True)

# Rimuove duplicati
errori_totali = errori_totali.drop_duplicates().reset_index(drop=True)

# Salva il file aggiornato con il nome corretto della colonna
errori_totali.to_csv(errori_path, index=False)

print("\n🔄 File Errori.csv aggiornato correttamente senza duplicare colonne.")
#playsound("Successo.mp3")

In [ ]:
list(comuni_da_analizzare.keys())[7:10]

In [ ]:
comune_dic = 'Barletta-Andria-Trani'
re.sub(r"[-' ]+", "", comune_dic.lower())

In [ ]:
inf_comunali = pd.read_csv("inf_comunali.csv")
#inf_comunali.head()

In [ ]:
inf_comunali[inf_comunali["Comune"] == "Fragagnano"]

## Cancellare le righe

In [ ]:
['Bitetto']

In [ ]:
import pandas as pd

In [ ]:
def cancellare():
    for comune in list(cancella.keys()):
        list_dfs = ['inf_comunali.csv', 'inf_cronologia_edifici.csv', 'inf_età.csv', 'inf_famiglie.csv', 'inf_interni.csv', 'inf_numero_di_piani.csv',
                    'inf_parrocchie.csv', 'inf_redditi_01.csv', 'inf_redditi_02.csv', 'inf_redditi_03.csv', 'inf_scolarizzazione.csv', 'inf_scuole.csv',
                    'inf_settori.csv', 'inf_stranieri.csv']

        for x in list_dfs:
            df = pd.read_csv(x)
            df = df.drop(df[df['Comune'] == comune].index)
            df.to_csv(x, index=False)
        
            informazioni_geografiche.loc[informazioni_geografiche["Comune"] == comune, "Raccolto"] = 0
            informazioni_geografiche.to_csv("informazioni_geografiche.csv", index=False)

#cancellare()

C:\Users\schit\AppData\Local\Temp\ipykernel_46200\3639449420.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(x)
C:\Users\schit\AppData\Local\Temp\ipykernel_46200\3639449420.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(x)
C:\Users\schit\AppData\Local\Temp\ipykernel_46200\3639449420.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(x)
C:\Users\schit\AppData\Local\Temp\ipykernel_46200\3639449420.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(x)
C:\Users\schit\AppData\Local\Temp\ipykernel_46200\3639449420.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(x)
C:\Users\schit\AppData\Local\Temp\ipykernel_46200\36394

In [ ]:
province_regioni = {
    "Agrigento": "Sicilia",
    "Alessandria": "Piemonte",
    "Ancona": "Marche",
    "Aosta": "Valle d'Aosta",
    "Arezzo": "Toscana",
    "Ascoli Piceno": "Marche",
    "Asti": "Piemonte",
    "Avellino": "Campania",
    "Bari": "Puglia",
    "Barletta-Andria-Trani": "Puglia",
    "Belluno": "Veneto",
    "Benevento": "Campania",
    "Bergamo": "Lombardia",
    "Biella": "Piemonte",
    "Bologna": "Emilia-Romagna",
    "Bolzano": "Trentino-Alto Adige",
    "Brescia": "Lombardia",
    "Brindisi": "Puglia",
    "Cagliari": "Sardegna",
    "Caltanissetta": "Sicilia",
    "Campobasso": "Molise",
    "Caserta": "Campania",
    "Catania": "Sicilia",
    "Catanzaro": "Calabria",
    "Chieti": "Abruzzo",
    "Como": "Lombardia",
    "Cosenza": "Calabria",
    "Cremona": "Lombardia",
    "Crotone": "Calabria",
    "Cuneo": "Piemonte",
    "Enna": "Sicilia",
    "Fermo": "Marche",
    "Ferrara": "Emilia-Romagna",
    "Firenze": "Toscana",
    "Foggia": "Puglia",
    "Forlì-Cesena": "Emilia-Romagna",
    "Frosinone": "Lazio",
    "Genova": "Liguria",
    "Gorizia": "Friuli Venezia Giulia",
    "Grosseto": "Toscana",
    "Imperia": "Liguria",
    "Isernia": "Molise",
    "L'Aquila": "Abruzzo",
    "La Spezia": "Liguria",
    "Latina": "Lazio",
    "Lecce": "Puglia",
    "Lecco": "Lombardia",
    "Livorno": "Toscana",
    "Lodi": "Lombardia",
    "Lucca": "Toscana",
    "Macerata": "Marche",
    "Mantova": "Lombardia",
    "Massa-Carrara": "Toscana",
    "Matera": "Basilicata",
    "Messina": "Sicilia",
    "Milano": "Lombardia",
    "Modena": "Emilia-Romagna",
    "Monza e della Brianza": "Lombardia",
    "Napoli": "Campania",
    "Novara": "Piemonte",
    "Nuoro": "Sardegna",
    "Oristano": "Sardegna",
    "Padova": "Veneto",
    "Palermo": "Sicilia",
    "Parma": "Emilia-Romagna",
    "Pavia": "Lombardia",
    "Perugia": "Umbria",
    "Pesaro e Urbino": "Marche",
    "Pescara": "Abruzzo",
    "Piacenza": "Emilia-Romagna",
    "Pisa": "Toscana",
    "Pistoia": "Toscana",
    "Pordenone": "Friuli Venezia Giulia",
    "Potenza": "Basilicata",
    "Prato": "Toscana",
    "Ragusa": "Sicilia",
    "Ravenna": "Emilia-Romagna",
    "Reggio Calabria": "Calabria",
    "Reggio Emilia": "Emilia-Romagna",
    "Rieti": "Lazio",
    "Rimini": "Emilia-Romagna",
    "Roma": "Lazio",
    "Rovigo": "Veneto",
    "Salerno": "Campania",
    "Sassari": "Sardegna",
    "Savona": "Liguria",
    "Siena": "Toscana",
    "Siracusa": "Sicilia",
    "Sondrio": "Lombardia",
    "Sud Sardegna": "Sardegna",
    "Taranto": "Puglia",
    "Teramo": "Abruzzo",
    "Terni": "Umbria",
    "Torino": "Piemonte",
    "Trapani": "Sicilia",
    "Trento": "Trentino-Alto Adige",
    "Treviso": "Veneto",
    "Trieste": "Friuli Venezia Giulia",
    "Udine": "Friuli Venezia Giulia",
    "Varese": "Lombardia",
    "Venezia": "Veneto",
    "Verbano-Cusio-Ossola": "Piemonte",
    "Vercelli": "Piemonte",
    "Verona": "Veneto",
    "Vibo Valentia": "Calabria",
    "Vicenza": "Veneto",
    "Viterbo": "Lazio"
}